In [1]:
%%time
import pandas as pd
import os
from datetime import date
from datetime import datetime 
import numpy as np
from sklearn.model_selection import train_test_split

direc = 'C:/Users/rvozzi/OneDrive - genommalabinternacional/Documentos/ECD/Seminario/TP/'

mercado = 'TAFIROL'

#Leo el archivo transformado
data = pd.read_csv(direc + mercado + '_' + 'data.csv')

C:\Users\rvozzi\anaconda3\lib\site-packages\IPython\core\magics\execution.py:1324: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code, glob, local_ns)


Wall time: 3min 56s


# 1. Separo los Datos (Train y Test) de los Datos a Predecir

In [2]:
# Separo los datos que genero con Monto = 0 y los guardo en pred_data
# El resto de los datos los uso para train y test
pred_data = data[data['Monto']==0]
data = data[data['Monto']>0]

# 2. Hago el Train Test Split

In [3]:
%%time
# Saco el cpa ya que no sirve para el modelo
indices = data[['cpa', 'Marca', 'Submarca', 'pres_desc', 'lab', 'Segmento',
               'PROVINCIA', 'LOCALIDAD', 'RUTA', 'VISITA']]

X = data.drop(['cpa', 'Marca', 'Submarca', 'pres_desc', 'lab', 'Segmento',
               'PROVINCIA', 'LOCALIDAD', 'RUTA', 'VISITA', 'Monto'], axis=1)
y = data['Monto']

X_train, X_test, y_train, y_test = train_test_split(
                                        X, y, 
                                        test_size = 0.25, 
                                        #stratify = data[data.filter(regex='^es_',axis=1).columns]
                                        )

Wall time: 49.2 s


In [4]:
# estas son las columnas que quiero estratificar, contienen la:
# ['Marca', 'Submarca', 'pres_desc', 'lab', 'Segmento', 'PROVINCIA', 'LOCALIDAD', 'RUTA', 'VISITA']
data[data.filter(regex='^es_',axis=1).columns]

,es__ACTRON 600 RAPIDA ACCION 10 caps.blandas.,es__ACTRON 600 RAPIDA ACCION 20 caps.blandas.,es__ACTRON PLUS caps.x 8,es__ACTRON RAPIDA ACCION 10 caps.blandas.x 400mg.,es__ACTRON RAPIDA ACCION 20 caps.blandas.x 400mg.,es__ALIDASE 500mg comp.x 10,es__ANAFLEX PLUS RA caps bl x 8,es__ANAFLEX PLUS RA caps.bl.x 16,es__ANAFLEX caps.blandas x 16,es__ANAFLEX caps.blandas x 8,...,es_ruta__LITORAL,es_ruta__MENDOZA,es_ruta__NEA,es_ruta__NOA NORTE,es_ruta__NOA SUR,es_ruta__PATAGONIA,es_ruta__ROSARIO,es_visita__NO ATENDIDAS,es_visita__NO VISITADA,es_visita__VISITADA
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
889277,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
889278,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
889279,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
889280,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


# 3. Entreno el Modelo

In [5]:
%%time
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators = 500, max_depth = 10, min_samples_split = 5, min_samples_leaf = 2)
rf.fit(X_train, y_train)

MemoryError: Unable to allocate 7.22 GiB for an array with shape (1645, 589371) and data type float64

# 4. Realizo las Predicciones

In [6]:
# En el set de testeo
y_test_pred = rf.predict(X_test)

# En el set de pred
y_pred_pred = rf.predict(pred_data.drop(['cpa', 'Marca', 'Submarca', 'pres_desc', 'lab', 'Segmento',
               'PROVINCIA', 'LOCALIDAD', 'RUTA', 'VISITA', 'Monto'], axis=1))

NotFittedError: This RandomForestRegressor instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

# 5. Valido el Modelo

In [ ]:
from sklearn.metrics import mean_squared_error
mse_rf = mean_squared_error(y_test, y_test_pred)
print(mse_rf)

In [ ]:
mse_rf**0.5

In [ ]:
y_test.mean()

In [ ]:
y_test.std()

In [ ]:
from sklearn.metrics import r2_score
r2_score(y_test, y_test_pred)

In [ ]:
resid_test = y_test-y_test_pred

In [ ]:
y_train_pred = rf.predict(X_train)
resid_train = y_train-y_train_pred
mean_squared_error(y_train, y_train_pred)**0.5

In [ ]:
y_train.mean()

In [ ]:
r2_score(y_train, y_train_pred)

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(y_test_pred, resid_test)
plt.show()

In [ ]:
resid_test.mean()

In [ ]:
(resid_test - resid_test.mean())/resid_test.std()

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(y_train_pred, resid_train)
plt.show()

In [ ]:
resid_train.mean()

In [ ]:
data['Monto'].std()

In [ ]:
"""
https://scikit-learn.org/stable/modules/model_evaluation.html#regression-metrics
3.3.4.12. Visual evaluation of regression models

#residuos vs predichos en entrenamiento
from sklearn.metrics import PredictionErrorDisplay
display = PredictionErrorDisplay(y_true=y_train, y_pred=y_train_pred, kind="residual_vs_predicted")
display.plot()
plt.show()

kind="actual_vs_predicted"

Para poder ejecutar esto ver 
https://scikit-learn.org/dev/developers/advanced_installation.html#install-bleeding-edge
"""

# 6. Exporto los Datos

In [ ]:
# Tengo , X_test, y_train, y_test, y_train_pred, y_test_pred

# Junto los datos de train
X_train['Monto'] = y_train
X_train['Monto_Predicho'] = y_train_pred
X_train.sort_index(inplace = True)

#Junto los Datos de testeo
X_test['Monto'] = y_test
X_test['Monto_Predicho'] = y_test_pred
X_test.sort_index(inplace = True)

#Junto los Datos a Predecir
pred_data['Monto_Predicho'] = y_pred_pred

X = pd.concat([X_train, X_test], ignore_index=False)
X.sort_index(inplace = True)

In [ ]:
X

In [ ]:
data['Monto_Predicho'] = X['Monto_Predicho']
data = data[['cpa', 'Marca', 'Submarca', 'pres_desc', 'lab', 'Segmento',
               'PROVINCIA', 'LOCALIDAD', 'RUTA', 'VISITA', 'Monto', 'Monto_Predicho']]
data

In [ ]:
pred_data = pred_data[['cpa', 'Marca', 'Submarca', 'pres_desc', 'lab', 'Segmento',
               'PROVINCIA', 'LOCALIDAD', 'RUTA', 'VISITA', 'Monto', 'Monto_Predicho']]
pred_data

In [ ]:
data = pd.concat([data, pred_data], ignore_index=False)
data.sort_index(inplace = True)
data['Oportunidad'] = data['Monto_Predicho'] - data['Monto']
data

In [ ]:
data.to_csv(path_or_buf = direc + 'pred_' + mercado + '.csv',
                    sep = ',', index = False)